In [2]:
import os, sys

# find repo root (looks for liars_poker/ or pyproject.toml)
def find_repo_root(start_dir: str) -> str:
    cur = os.path.abspath(start_dir)
    for _ in range(6):
        if os.path.isdir(os.path.join(cur, "liars_poker")) or os.path.exists(os.path.join(cur, "pyproject.toml")):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    return os.path.abspath(os.path.join(start_dir, "..", ".."))

NB_DIR = os.getcwd()
REPO_ROOT = find_repo_root(NB_DIR)
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

ARTIFACTS_ROOT = os.path.join(REPO_ROOT, "artifacts")
os.makedirs(ARTIFACTS_ROOT, exist_ok=True)

print("repo root   :", REPO_ROOT)
print("artifacts   :", ARTIFACTS_ROOT)


repo root   : c:\Users\adidh\Documents\liars_poker
artifacts   : c:\Users\adidh\Documents\liars_poker\artifacts


In [ ]:
from liars_poker import (
    GameSpec, Env, InfoSet, Rules,
    Policy, TabularPolicy, CommitOnceMixture, RandomPolicy,
    eval_both_seats
)

from liars_poker.training.fsp import fsp_loop, dense_fsp_loop, plot_exploitability_series

from liars_poker.algo.br_exact import best_response_exact
from liars_poker.algo.br_mc import best_response_mc

from liars_poker.algo.br_exact_dense_to_dense import best_response_exact as best_response_denser

from liars_poker.core import possible_starting_hands
from liars_poker.policies.tabular_dense import DenseTabularPolicy
from liars_poker.serialization import save_policy

In [ ]:
spec = GameSpec(ranks=3, suits=4, hand_size=3, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True)
rules = Rules(spec)

In [ ]:
all_specs = [
    GameSpec(ranks=2, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True),
    GameSpec(ranks=2, suits=2, hand_size=1, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=2, suits=3, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=2, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=2, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair", "Trips"), suit_symmetry=True),

    GameSpec(ranks=3, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True),
    GameSpec(ranks=3, suits=2, hand_size=1, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=3, suits=3, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=3, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=3, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair", "Trips"), suit_symmetry=True),

    GameSpec(ranks=4, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True),
    GameSpec(ranks=4, suits=2, hand_size=1, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=4, suits=3, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=4, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=4, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair", "Trips"), suit_symmetry=True),

    GameSpec(ranks=5, suits=2, hand_size=1, claim_kinds=("RankHigh"), suit_symmetry=True),
    GameSpec(ranks=5, suits=2, hand_size=1, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=5, suits=3, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=5, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=5, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair", "Trips"), suit_symmetry=True),

    GameSpec(ranks=6, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True),
    GameSpec(ranks=6, suits=2, hand_size=1, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=6, suits=3, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=6, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),


    GameSpec(ranks=7, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True),
    GameSpec(ranks=7, suits=2, hand_size=1, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=7, suits=3, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),
    GameSpec(ranks=7, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True),


]

In [ ]:
for spec in all_specs:
    pol, info = dense_fsp_loop(spec, 1000)
    plot_exploitability_series(info)

    # TODO: save pol, and some of info

In [ ]:
from datetime import datetime
from pathlib import Path
import json

RUN_ROOT = Path(ARTIFACTS_ROOT) / 'benchmark_runs'
RUN_ROOT.mkdir(parents=True, exist_ok=True)

def save_run(run_id: str, policy: Policy, info: dict, spec: GameSpec):
    run_dir = RUN_ROOT / run_id
    (run_dir / 'policy').mkdir(parents=True, exist_ok=True)
    # Save policy
    save_policy(policy, run_dir / 'policy')
    # Save metrics
    metrics = {
        'spec': spec.to_json(),
        'exploitability_series': info.get('exploitability_series', []),
        'p_values': info.get('p_values', []),
        'timestamp': datetime.utcnow().isoformat() + 'Z',
    }
    (run_dir / 'metrics.json').write_text(json.dumps(metrics, indent=2), encoding='utf-8')
    print(f'Saved run to {run_dir}')

# # Example: save the last run if available
# try:
#     save_run('last_run', pol, info, spec)
# except Exception as exc:
#     print('Save skipped:', exc)
